In [4]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [5]:
df = pd.read_pickle("./dimapac.pkl")


In [6]:
def ip_from_string(ips):
    return "".join(chr(int(n)) for n in ips.split("."))

In [7]:
df_deopped = df.drop(['route', 'timestamp'], axis=1)

In [8]:
df_deopped.iloc[:3800]


,client_package_size_mean,client_package_size_std,server_package_size_mean,server_package_size_std,client_batch_sizes_mean,client_batch_sizes_std,server_batch_sizes_mean,server_batch_sizes_std,client_batch_counts_mean,server_batch_counts_mean,...,client_package_size_sum,client_application_size_sum,client_package_count,client_batch_counts_sum,server_package_size_sum,server_application_size_sum,server_package_count,server_batch_counts_sum,transport_protocol,ip_protocol_version
0,184.428571,203.200933,605.571429,694.721972,525.500000,45.500000,2005.500000,615.500000,1.500000,1.500000,...,1291,889.0,7,2,4239,3849.0,7,2,6,4
1,110.285714,111.966832,609.500000,713.942225,364.000000,0.000000,3451.000000,0.000000,1.000000,3.000000,...,772,298.0,7,1,3657,3253.0,6,1,6,4
8,285.000000,481.936977,529.785714,627.894428,1405.000000,1464.657184,2342.333333,1408.180939,2.000000,2.333333,...,4845,3891.0,17,3,7417,6649.0,14,3,6,4
10,76.500000,16.944025,65.500000,19.937402,195.000000,0.000000,165.000000,0.000000,2.000000,2.000000,...,459,63.0,6,1,393,45.0,6,1,6,4
19,391.333333,423.441384,422.090909,439.393094,939.200000,1138.449718,773.833333,1092.540376,2.400000,1.833333,...,4696,4164.0,12,5,4643,4181.0,11,6,17,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9029,75.454545,23.779670,67.000000,14.387495,416.000000,0.000000,48.500000,48.500000,4.000000,0.500000,...,830,152.0,11,1,469,31.0,7,2,6,4
9035,155.720000,288.676942,372.000000,512.652134,722.750000,708.632618,2283.333333,1746.946542,2.500000,3.666667,...,3893,2231.0,25,4,7440,6136.0,20,3,6,4
9036,116.809524,112.336871,165.000000,123.772925,226.142857,191.380644,408.076923,334.994480,1.142857,1.692308,...,4906,2110.0,42,14,6435,3853.0,39,13,6,4
9037,165.250000,201.873104,490.333333,639.273025,360.750000,233.897814,1362.333333,1183.808074,1.000000,1.333333,...,1983,1179.0,12,4,4413,3823.0,9,3,6,4


In [11]:
train_length = int(df_deopped.shape[0] * 0.8)
train_df = df_deopped.iloc[:train_length]
test_df = df_deopped.iloc[train_length:]

nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
parameters = {'kernel':['linear', 'rbf'], 'gamma' : gammas, 'nu': nus}
svc = OneClassSVM()
svc_clf = GridSearchCV(svc, parameters, scoring='f1', return_train_score=True)
svc_clf.fit(train_df, np.full(train_length, 1))

GridSearchCV(estimator=OneClassSVM(),
             param_grid={'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['rbf'],
                         'nu': [0.001, 0.01, 0.1, 1]},
             return_train_score=True, scoring='f1')

In [12]:
print(svc_clf.best_params_)

{'gamma': 0.01, 'kernel': 'rbf', 'nu': 0.001}


In [107]:
y = svc_clf.predict(my1601)

In [13]:
extern_traffic = pd.read_pickle("../traffic_analyzer/file.pkl")
extern_traffic = extern_traffic.drop(['route', 'timestamp'], axis=1)

In [14]:
print("Класс '-1' - нехарактерный траффик, '1' - характерный траффик")

print("\n---Информация о пакетах взятых из открытых источников и никак не связанных с начаьной выборкой ")
y = svc_clf.predict(extern_traffic)
uniqueValues, occurCount = np.unique(y, return_counts=True)
print("Полученные классы : " , uniqueValues[::-1])
print("Количество обхектов в каждом классе соотвественно : ", occurCount)

print("\n---Информация о пакетах которые были отложены из начальный выборки для теста и в которых были сымитированы угрозы")
y = svc_clf.predict(test_df)
uniqueValues, occurCount = np.unique(y, return_counts=True)
print("Полученные классы : " , uniqueValues[::-1])
print("Количество обхектов в каждом классе соотвественно : ", occurCount)

Класс '-1' - нехарактерный траффик, '1' - характерный траффик

---Информация о пакетах взятых из открытых источников и никак не связанных с начаьной выборкой 
Полученные классы :  [-1]
Количество обхектов в каждом классе соотвественно :  [1175]

---Информация о пакетах которые были отложены из начальный выборки для теста и в которых были сымитированы угрозы
Полученные классы :  [ 1 -1]
Количество обхектов в каждом классе соотвественно :  [760 178]


In [114]:
760 / (760 + 178)

0.8102345415778252

In [9]:
import matplotlib
from matplotlib import pyplot as 
from matplotlib.pyplot import figure

In [10]:
def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [104]:
figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
f_importances(svc_clf.best_estimator_.coef_[0], FEATURES)

AttributeError: coef_ is only available when using a linear kernel

<Figure size 960x960 with 0 Axes>

In [105]:
svc_clf.best_estimator_.coef_[0]

AttributeError: coef_ is only available when using a linear kernel

In [95]:
from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(25, input_dim=df_deopped.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(df_deopped.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(df_deopped,df_deopped,verbose=1,epochs=100)

Epoch 1/100
4687/4687 [==============================] - 1s 142us/sample - loss: nan
Epoch 2/100
4687/4687 [==============================] - 0s 61us/sample - loss: nan
Epoch 3/100
4687/4687 [==============================] - 0s 57us/sample - loss: nan
Epoch 4/100
4687/4687 [==============================] - 0s 59us/sample - loss: nan
Epoch 5/100
4687/4687 [==============================] - 0s 59us/sample - loss: nan
Epoch 6/100
4687/4687 [==============================] - 0s 56us/sample - loss: nan
Epoch 7/100
4687/4687 [==============================] - 0s 55us/sample - loss: nan
Epoch 8/100
4687/4687 [==============================] - 0s 55us/sample - loss: nan
Epoch 9/100
4687/4687 [==============================] - 0s 54us/sample - loss: nan
Epoch 10/100
4687/4687 [==============================] - 0s 55us/sample - loss: nan
Epoch 11/100
4687/4687 [==============================] - 0s 54us/sample - loss: nan
Epoch 12/100
4687/4687 [==============================] - 0s 54us/sample 

In [92]:
pred = model.predict(df_deopped)
score1 = np.sqrt(metrics.mean_squared_error(pred,df_deopped))
# pred = model.predict(x_normal)
# score2 = np.sqrt(metrics.mean_squared_error(pred,x_normal))
# pred = model.predict(x_attack)
# score3 = np.sqrt(metrics.mean_squared_error(pred,x_attack))
# print(f"Out of Sample Score (RMSE): {score1}")
# print(f"Insample Normal Score (RMSE): {score2}")
# print(f"Attack Underway Score (RMSE): {score3}")
score1

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=encdf.shape[1], activation='relu'))
model.add(Dense(50, input_dim=encdf.shape[1], activation='relu'))
model.add(Dense(10, input_dim=encdf.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto')
model.fit(encdf,encdf,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=1000)

In [89]:
encdf= df_deopped

In [88]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


In [90]:
encode_numeric_zscore(encdf, 'client_package_size_mean')
encode_numeric_zscore(encdf, 'client_package_size_std') #2
encode_numeric_zscore(encdf, 'server_package_size_mean') #3
encode_numeric_zscore(encdf, 'server_package_size_std') #4
encode_numeric_zscore(encdf, 'client_batch_sizes_mean') #5
encode_numeric_zscore(encdf, 'client_batch_sizes_std') #6
encode_numeric_zscore(encdf, 'server_batch_sizes_mean') #7
encode_numeric_zscore(encdf, 'server_batch_sizes_std') #8
encode_numeric_zscore(encdf, 'client_batch_counts_mean') #9
encode_numeric_zscore(encdf, 'server_batch_counts_mean') #10
encode_numeric_zscore(encdf, 'client_efficiency') #11
encode_numeric_zscore(encdf, 'server_efficiency') #12
encode_numeric_zscore(encdf, 'ratio_sizes') #13
encode_numeric_zscore(encdf, 'ratio_application_size') #14
encode_numeric_zscore(encdf, 'ratio_packages') #15
encode_numeric_zscore(encdf, 'client_package_size_sum') #16
encode_numeric_zscore(encdf, 'client_application_size_sum')#17
encode_numeric_zscore(encdf, 'client_package_count') #18
encode_numeric_zscore(encdf, 'client_batch_counts_sum') #19
encode_numeric_zscore(encdf, 'server_package_size_sum') #20
encode_numeric_zscore(encdf, 'server_application_size_sum') #21
encode_numeric_zscore(encdf, 'server_package_count')#22
encode_numeric_zscore(encdf, 'server_batch_counts_sum') #23
encode_numeric_zscore(encdf, 'transport_protocol') #24
encode_numeric_zscore(encdf, 'ip_protocol_version') #25


In [11]:
FEATURES = [
    'client_package_size_mean', #1
    'client_package_size_std', #2
    'server_package_size_mean', #3
    'server_package_size_std', #4
    'client_batch_sizes_mean', #5
    'client_batch_sizes_std', #6
    'server_batch_sizes_mean', #7
    'server_batch_sizes_std', #8
    'client_batch_counts_mean', #9
    'server_batch_counts_mean', #10
    'client_efficiency', #11
    'server_efficiency', #12
    'ratio_sizes', #13
    'ratio_application_size', #14
    'ratio_packages', #15
    'client_package_size_sum', #16
    'client_application_size_sum', #17
    'client_package_count', #18
    'client_batch_counts_sum', #19
    'server_package_size_sum', #20
    'server_application_size_sum', #21
    'server_package_count', #22
    'server_batch_counts_sum', #23
    'transport_protocol', #24
    'ip_protocol_version', #25
]

In [4]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
from sklearn.datasets import make_classification

In [5]:
train_length = int(df_deopped.shape[0] * 0.8)
train_df = df_deopped.iloc[:train_length]
test_df = df_deopped.iloc[train_length:]

In [ ]:
clf = IsolationForest(random_state=47)

param_grid = {'n_estimators': list(range(50, 100, 5)), 
              'max_samples': list(range(40, 80, 5)), 
              'contamination': [0.1, 0.2, 0.3, 0.4, 0.5], 
              'max_features': [5,10,15], 
              'bootstrap': [True, False], 
              'n_jobs': [5, 10, 20, 30]}
grid_dt_estimator = model_selection.GridSearchCV(clf, 
                                                 param_grid,
                                                 scoring="f1", 
                                                 refit=True,
                                                 cv=10, 
                                                 return_train_score=True)
grid_dt_estimator.fit(train_df, np.full(train_length, 1))

/home/theeska/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/theeska/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/theeska/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
train_df.shape